In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

## Load model and test data

In [ ]:
experiment_relative_path = r"outputs\2025-08-06_09-53-26_eda_shear_50M"

eval_data_path: Path = Path(r"C:\Users\cervinka\cervinka\GitHub\MathCAS\datasets\dataset_compressible_flow_5M_test.csv")

In [ ]:
# Load m experiment directory #

import torch
from pathlib import Path
from config import load_config, FullConfig
from src.architecture import NeuralNetwork

exp_dir: Path = project_root / Path(experiment_relative_path)

config_path = exp_dir / "config.yaml"

config: FullConfig = load_config(config_path)

## Make predictions on the test data

In [ ]:
from src.evaluation import evaluate

evaluate(config, exp_dir, eval_data_path)

## Double model Evaluation

In [ ]:
import pandas as pd
import torch
from src.evaluation import load_model
from config import load_config
from utils.metrics import compute_regression_metrics

# Paths for models and configs
shear_exp_dir = project_root / r"outputs\2025-08-26_12-47-54_shear_dan_750"
shear_config = load_config(shear_exp_dir / "config.yaml")
shear_model = load_model(shear_config, shear_exp_dir)

res_exp_dir = project_root / r"outputs\2025-08-27_11-17-55_res_trueShear_dan_750"
res_config = load_config(res_exp_dir / "config.yaml")
res_model = load_model(res_config, res_exp_dir)

# Load data
df = pd.read_csv(r"C:\Users\cervinka\cervinka\GitHub\MathCAS\datasets\dataset_compressible_flow_5M_test.csv")

# Predict shear
shear_inputs = df[shear_config.data.in_cols].values
shear_tensor = torch.tensor(shear_inputs, dtype=torch.float32)
with torch.no_grad():
    pred_shear = shear_model(shear_tensor).squeeze().numpy()
df["PredShear"] = pred_shear

# Prepare inputs for second model (original + PredShear)
res_inputs = df[res_config.data.in_cols].values
res_tensor = torch.tensor(res_inputs, dtype=torch.float32)
with torch.no_grad():
    res_pred = res_model(res_tensor).squeeze().numpy()

# Print metrics for the second model
output_cols = res_config.data.out_cols
y_true = df[output_cols].values
y_pred = res_pred if res_pred.ndim == 2 else res_pred.reshape(-1, 1)

for i, col in enumerate(output_cols):
    y_true_col = y_true[:, i]
    y_pred_col = y_pred[:, i]
    metrics = compute_regression_metrics(y_true_col, y_pred_col)
    print(f"\nMetrics for output '{col}':")
    for k, v in metrics.items():
        print(f"  {k}: {v}")


Metrics for output 'ResVort':
  MAPE: 2.2642e+01
  MAE: 1.8322e-03
  MSE: 1.3190e-05
  MedAE: 1.9151e-04
  MaxAE: 2.4958e-01

Metrics for output 'ResStrain':
  MAPE: 9.3281e+00
  MAE: 3.1150e-03
  MSE: 2.1701e-05
  MedAE: 2.3877e-03
  MaxAE: 4.2955e-01


: 